In [45]:
import PIL.Image as img
import numpy as np
import csv
import tensorflow as tf

In [66]:
INPUT_RES = 32
learning_rate = 0.001
names = ['rice', 'candy', 'jam', 'coffee', 'vinegar', 'chocolate', 'sugar', 'water', 'juice', 'milk', 'soda', 'nuts', 'chips', 'spices', 'cereal', 'beans', 'cake', 'honey', 'flour', 'pasta', 'tomatosauce', 'tea', 'corn', 'oil', 'fish']
print(len(names))

25


In [98]:
def readTrainData():
    size = INPUT_RES, INPUT_RES
    dataFiles = csv.reader(open('./data/train.csv'), delimiter=',')
    data = []
    labels = []
    for imName, label in dataFiles:
        if imName == 'image_id':
            continue
        im = img.open('./data/train_img/' + imName + '.png').convert('L').resize(size, img.ANTIALIAS)
        data.append(np.asarray(im).reshape(32*32))
        labels.append(names.index(label)+1)
        
    return np.asarray(data), labels
    
    
    
#     image = img.open('data/train_img/train_2a.png')
#     size = INPUT_RES, INPUT_RES
#     image = image.convert('L').resize(size, img.ANTIALIAS)
#     #print(image.split()[0])
#     print(image)
#     #image.show()
#     print(np.asarray(image).shape)
    
data, labels = readTrainData()

In [113]:
# print(data.shape)
# print(len(labels))\
# 26 classes
NCLASSES = 25

x = tf.placeholder(tf.float32, [None, 32 * 32])
y = tf.placeholder(tf.float32, [None, NCLASSES])
W1 = tf.Variable(tf.random_normal([5, 5, 1, 64]))
b1 = tf.Variable(tf.random_normal([64]))
W2 = tf.Variable(tf.random_normal([5, 5, 64, 64]))
b2 = tf.Variable(tf.random_normal([64]))
W3 = tf.Variable(tf.random_normal([8*8*64, 1024]))
b3 = tf.Variable(tf.random_normal([1024]))
W_out = tf.Variable(tf.random_normal([1024, NCLASSES]))
b_out = tf.Variable(tf.random_normal([NCLASSES]))

In [114]:
def conv_layer(x, W, b):
    conv = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    conv_with_b = tf.nn.bias_add(conv, b)
    conv_out = tf.nn.relu(conv_with_b)
    return conv_out


def maxpool_layer(conv, k=2):
    return tf.nn.max_pool(conv, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [115]:
def model():
    #x_reshaped = tf.reshape(x, shape=[-1, 32, 32, 1])
    x_reshaped = tf.reshape(x, shape=[-1, 32, 32, 1])
    conv_out1 = conv_layer(x_reshaped, W1, b1)
    maxpool_out1 = maxpool_layer(conv_out1)
    norm1 = tf.nn.lrn(maxpool_out1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    conv_out2 = conv_layer(norm1, W2, b2)
    norm2 = tf.nn.lrn(conv_out2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    maxpool_out2 = maxpool_layer(norm2)

    maxpool_reshaped = tf.reshape(maxpool_out2, [-1, W3.get_shape().as_list()[0]])
    local = tf.add(tf.matmul(maxpool_reshaped, W3), b3)
    local_out = tf.nn.relu(local)

    out = tf.add(tf.matmul(local_out, W_out), b_out)
    return out

In [116]:
model_op = model()

cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=model_op, labels=y)
)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(model_op, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [118]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    onehot_labels = tf.one_hot(labels, len(names), on_value=1., off_value=0., axis=-1)
    #print(onehot_labels)
    onehot_vals = sess.run(onehot_labels)
    batch_size = len(data) // 200
    print('batch size', batch_size)
    for j in range(0, 1000):
        avg_accuracy_val = 0.
        batch_count = 0.
        for i in range(0, len(data), batch_size):
            batch_data = data[i:i+batch_size, :]
            #print(batch_data.shape)
            batch_onehot_vals = onehot_vals[i:i+batch_size, :]
            #print(batch_onehot_vals.shape)
            _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x: batch_data, y: batch_onehot_vals})
            avg_accuracy_val += accuracy_val
            batch_count += 1.
        avg_accuracy_val /= batch_count
        print('Epoch {}. Avg accuracy {}'.format(j, avg_accuracy_val))


batch size 16
Epoch 0. Avg accuracy 0.05881011612083188
Epoch 1. Avg accuracy 0.07837893865743087
Epoch 2. Avg accuracy 0.09425787731486174
Epoch 3. Avg accuracy 0.10638474298650352
Epoch 4. Avg accuracy 0.1343490879111622
Epoch 5. Avg accuracy 0.14210199004975124
Epoch 6. Avg accuracy 0.15366915431781789
Epoch 7. Avg accuracy 0.14769900497512436
Epoch 8. Avg accuracy 0.16050995033771837
Epoch 9. Avg accuracy 0.19036069660637508
Epoch 10. Avg accuracy 0.18194444447904085
Epoch 11. Avg accuracy 0.19716003318479405
Epoch 12. Avg accuracy 0.20400082920469456
Epoch 13. Avg accuracy 0.2161898839829573
Epoch 14. Avg accuracy 0.22796434497655327
Epoch 15. Avg accuracy 0.23667081263824483
Epoch 16. Avg accuracy 0.2541252073660419
Epoch 17. Avg accuracy 0.272802653597362
Epoch 18. Avg accuracy 0.2730928690575842
Epoch 19. Avg accuracy 0.2712064677506537
Epoch 20. Avg accuracy 0.2908167496545991
Epoch 21. Avg accuracy 0.2826907131357573
Epoch 22. Avg accuracy 0.3056799336823065
Epoch 23. Avg acc

Epoch 193. Avg accuracy 0.6569029851932431
Epoch 194. Avg accuracy 0.6771351579409927
Epoch 195. Avg accuracy 0.6930140962648155
Epoch 196. Avg accuracy 0.6771766172712715
Epoch 197. Avg accuracy 0.6955431180806896
Epoch 198. Avg accuracy 0.7039593701338887
Epoch 199. Avg accuracy 0.702529021756566
Epoch 200. Avg accuracy 0.6703150915862316
Epoch 201. Avg accuracy 0.6911898842498437
Epoch 202. Avg accuracy 0.7129353237389332
Epoch 203. Avg accuracy 0.6935530681515214
Epoch 204. Avg accuracy 0.7160240468101122
Epoch 205. Avg accuracy 0.6799543947722781
Epoch 206. Avg accuracy 0.6674958544584056
Epoch 207. Avg accuracy 0.7070273635399282
Epoch 208. Avg accuracy 0.7188018246076593
Epoch 209. Avg accuracy 0.6861525708763161
Epoch 210. Avg accuracy 0.6992537316991322
Epoch 211. Avg accuracy 0.652922885927988
Epoch 212. Avg accuracy 0.6908582093110726
Epoch 213. Avg accuracy 0.6790422889130625
Epoch 214. Avg accuracy 0.6991915424071734
Epoch 215. Avg accuracy 0.7281716421468934
Epoch 216. Av

Epoch 385. Avg accuracy 0.6289179105663774
Epoch 386. Avg accuracy 0.6575663350707855
Epoch 387. Avg accuracy 0.6606135988116857
Epoch 388. Avg accuracy 0.6520107797722319
Epoch 389. Avg accuracy 0.6619402988633113
Epoch 390. Avg accuracy 0.6214552239992133
Epoch 391. Avg accuracy 0.6697139307041073
Epoch 392. Avg accuracy 0.6930140965613559
Epoch 393. Avg accuracy 0.6513474295981487
Epoch 394. Avg accuracy 0.6657338308457711
Epoch 395. Avg accuracy 0.6668117746191832
Epoch 396. Avg accuracy 0.6373134329544371
Epoch 397. Avg accuracy 0.6392620235533264
Epoch 398. Avg accuracy 0.6255804315135254
Epoch 399. Avg accuracy 0.6510364846210575
Epoch 400. Avg accuracy 0.6503938644086543
Epoch 401. Avg accuracy 0.6366915424071734
Epoch 402. Avg accuracy 0.6622305144718037
Epoch 403. Avg accuracy 0.6367122723688534
Epoch 404. Avg accuracy 0.6520107797722319
Epoch 405. Avg accuracy 0.6404436156524355
Epoch 406. Avg accuracy 0.688971808004142
Epoch 407. Avg accuracy 0.6796641794603262
Epoch 408. A

KeyboardInterrupt: 